In [193]:
import copy


class WumpusPuzzle:
    def __init__(self, s, a, q, qa, p):
        self.size = s
        self.arrows = a
        self.query = q
        self.queryAnswer = qa
        self.path = p

In [194]:
class Clause:
    def __init__(self, l, c):
        #list of literals
        self.literals = l
        #booleans
        self.isDisjunction = not c
        self.isConjunction = c

In [195]:
class Literal:
    def __init__(self, p, n):
        #boolean
        self.negation = n
        #predicate
        self.predicate = p

    def getValue(self):
        if(self.predicate.truthValue == None):
            return None
        if self.negation:
            return self.predicate.truthValue
        else:
            return not self.predicate.truthValue


In [196]:
def literalsCancel(literal1, literal2):
    if((literal1.predicate.equalsOther(literal2.predicate)) and (literal1.negation != literal2.negation)):
        return True
    else:
        return False


In [197]:
class Predicate:
    def __init__(self, v, func):
        self.variables = v
        self.function = func
        self.truthValue = func(v)

    def equalsOther(self, otherPredicate):
        if(self.variables == otherPredicate.variables and self.function == otherPredicate.function):
            return True
        return False


In [198]:
def Cat(value):
    if(value[0].name == 'Tuna'):
        return True
    if(value[0].name == None):
        return None
    return False


In [199]:
def Loves(values):
    if(values[1] in values[0].loves):
        return True
    return False


In [200]:
def Animal(values):
    if(values[0].isAnimal):
        return True
    return False

In [201]:
def Kills(values):
    if(values[1] in values[0].kills):
        return True
    return False

In [202]:
class toyVariable:
    def __init__(self, n, a, l, k):
        #string
        self.name = n
        #boolean
        self.isAnimal = a
        #lists
        self.loves = l
        self.kills = k


In [203]:
def resolution(KB, query):
    clauses = copy.deepcopy(KB)
    clauses.append(query)
    new = set()
    while(True):
        clausePairs = set()
        #setting up all pairs of clauses
        for clause in clauses:
            for clause2 in clauses:
                if(clause != clause2):
                    clausePairs.add([clause, clause2])
        for pair in clausePairs:
            resolvents = resolve(pair[0], pair[1])
            if(resolvents == None):
                return True
            else:
                new.add(resolvents)
        if(new.issubset(clauses)):
            return False
        clauses = clauses.union(new)


def resolve(clause1, clause2):
    returnedClauses = []
    for literal1 in clause1.literals:
        for literal2 in clause2.literals:
            if(literalsCancel(literal1,literal2)):
                clause1.literals.remove(literal1)
                clause2.literals.remove(literal2)
    returnedClauses.append(clause1)
    returnedClauses.append(clause2)






In [204]:
PuzzleFile = 'Caves/_wumpus_paths_/easy/path_e3.txt'

def processFile(fileName):
    with open(fileName, 'r', encoding='utf-8-sig') as file:
        content = file.read().strip()
        result = content.split("\n")
        result2 = result[0].split("GRID: ")
        xAxis = int(result2[1].split("x")[0])
        yAxis = int(result2[1].split("x")[1])
        size = [xAxis, yAxis]
        result3 = result[1].split("ARROWS: ")
        arrows = int(result3[1])
        index = 3
        path = result[index]
        pathArr = []
        while(path != ''):
            pathStr = path.strip().split("(")[1].split(")")
            xVal = int(pathStr[0].split(",")[0])
            yVal = int(pathStr[0].split(",")[1])
            breezeVal = pathStr[1].split("B:")[1][0]
            stenchVal = pathStr[1].split("S:")[1][0]
            pathVal = [(xVal, yVal), breezeVal, stenchVal]
            pathArr.append(pathVal)
            index+=1
            path = result[index]
        queryResult = result[index+1].split("QUERY: ")[1].split(',')
        query = [int(queryResult[0][1:]), int(queryResult[1][:-1])]
        resolution = result[index+2].split("RESOLUTION:")[1].strip()
        puzzle = WumpusPuzzle(size, arrows, query, resolution, pathArr)
        print(size)
        print(arrows)
        print(pathArr)
        print(query)
        print(resolution)
        return puzzle

puzzle = processFile(PuzzleFile)

[20, 20]
2
[[(0, 0), 'F', 'F'], [(1, 0), 'F', 'T'], [(2, 0), 'F', 'F'], [(2, 1), 'F', 'T'], [(3, 1), 'F', 'T'], [(4, 1), 'F', 'F'], [(4, 2), 'F', 'T'], [(5, 2), 'F', 'F'], [(5, 3), 'T', 'F'], [(4, 3), 'F', 'F'], [(3, 3), 'T', 'T'], [(2, 3), 'F', 'F'], [(2, 2), 'F', 'T']]
[1, 2]
SAFE


In [205]:
KB = set()

Tuna = toyVariable("Tuna", None, [], [])
Jack = toyVariable("Jack", None, [], [])
Curiosity = toyVariable("Curiosity", None, [], [])
x = toyVariable(None, True, [], [])
fx = toyVariable(None, True, [], [])
gx = toyVariable(None, True, [], [])
y = toyVariable(None, True, [], [])
z = toyVariable(None, True, [], [])



p = Predicate([Tuna], Cat)
l = Literal(p, True)
rule6 = Clause([l], False)

p = Predicate([x], Cat)
l = Literal(p, False)
p = Predicate([x], Animal)
l2 = Literal(p, True)
rule7 = Clause([l,l2], False)

p = Predicate([fx], Animal)
l = Literal(p, True)
p = Predicate([gx, x], Loves)
l2 = Literal(p, True)
rule1 = Clause([l,l2], False)


p = Predicate([x, fx], Loves)
l = Literal(p, False)
p = Predicate([gx, x], Loves)
l2 = Literal(p, True)
rule2 = Clause([l,l2], False)

p = Predicate([y, x], Loves)
l = Literal(p, False)
p = Predicate([z], Animal)
l2 = Literal(p, False)
p = Predicate([x,z], Kills)
l3 = Literal(p, False)
rule3 = Clause([l,l2,l3], False)

p = Predicate([x], Animal)
l = Literal(p, False)
p = Predicate([Jack, x], Loves)
l2 = Literal(p, True)
rule4 = Clause([l,l2], False)

p = Predicate([Jack,Tuna], Kills)
l = Literal(p, True)
p = Predicate([Curiosity,Tuna], Kills)
l2 = Literal(p, True)
rule5 = Clause([l,l2], False)

KB.add(rule1)
KB.add(rule2)
KB.add(rule3)
KB.add(rule4)
KB.add(rule5)
KB.add(rule6)
KB.add(rule7)


#Testing literal cancel checking function
p = Predicate([Jack,Tuna], Kills)
l = Literal(p, True)
p = Predicate([Jack,Tuna], Kills)
l2 = Literal(p, False)
print(literalsCancel(l,l2))

True
